# Calculate GWP and create tables

The script calculates the GWP of hydrogen and methane using a steady-state perturbation approach.

**Simulations**:
- **CTRL**: fixed surface concentration of hydrogen and methane
- **10H2**: as CTRL, but with surface H2 increased by 10%. Run to steady state.
- **10CH4**: as CTRL, but with surface CH4 increased by 10%. Run to steady state.

In addition, two models have perturbed H2 emissions instead of the surface mole fraction of H2.

GWP is the ratio of the absolute global warming potential (AGWP) for hydrogen to that for CO2. AGWP is defined as the time-integrated radiative forcing to a 1 kg pulse emission of that gas over a given time horizon, here 100 years. 

AGWP is equal to the steady-state radiative forcing (W m-2) divided by the delta flux (Tg-H2 yr-1) for a perturbed vs control. This delta flux between the perturbed (10H2 or 10CH4) and the control simulation (CTRL) includes chemical feedbacks. 

Based on the three set of simulations, we calculate the radiative forcing in the perturbed relative to the control. As the models run with fixed surface concentration of methane, we need to have a separate perturbed methane run. We calculate the radiative forcing per Tg-CH4 (including the feedbacks, as feedbacks are included in the delta flux) for the methane perturbed run, and map the changes in the methane loss in the hydrogen perturbation with the results from the methane perturbed run. 

*GFDL-emi simulations from Paulot et al. (2021) with H2 emissions (200 Tg/yr).*

#### OUTLINE:
**PART I: Read model results**

**PART II: GWP calculations**

**PART III: Main results and tabels**

**Appendix with additional results**

**Uncertainty analysis output**

This is the main notebook for the GWP calculations. The uncertainty analysis and plotting are done in separate notebooks. 

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x) if abs(x)<0 else ('{:,.2f}'.format(x) if abs(x)<10 else ('{:,.1f}'.format(x) if abs(x)<100 else '{:,.0f}'.format(x))))

Input and output path:

In [2]:
path = r"./input/"
outputpath= r"./output/"

Constants:

In [3]:
#AGWP100_CO2 [mW yr m-2 Tg-1] Source: Table 7.SM.6 in IPCC AR6: 0.0895 pW m-2 yr kg-1 (p=10^-12) 
agwp100_CO2 = 0.0895


#CH4 tau_strat[yr] 
tau_strat = 120.0*2.0 #Multiplied by to for not double counting OH loss in stratosphere. 

#CH4 tau_soil [yr] 
tau_soil = 160.0

#Specific RF for CH4 [mW m-2 ppb-1] Etminan et al., 2016
spec_rf_ch4 = 0.44800

#The adjustment is –14% ± 15% IPCC AR6
spec_rf_ch4 = spec_rf_ch4*(1.0-0.14)
print(spec_rf_ch4)

0.38528


**Dry deposition adjustments:** As there is large uncertanty in dry deposition, here is a possibility to spesify the soil sink value. The perturbations are adjusted by the same relative factors as in the control. 

In [4]:
adjust_drydep = False
if(adjust_drydep):
    drydep = 59.0
    outputpath = outputpath + 'drydep_'+ f'{drydep:.0f}_'

# Part I: Read model results

In this part, model results are read from the input files.

For GFDL-emi, 10H2 is the hydrogen only perturbed run, while 10CH4 is the hydrogen plus methane perturbed run.

## 1. Hydrogen budget

### 1.1 H2 burden [Tg]:

In [5]:
file = 'H2_burden.txt'
df_h2_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_burden.loc['10H2']-df_h2_burden.loc['CTRL']
delta.name = 'deltaH2'
df_h2_burden = df_h2_burden.append(delta)
df_h2_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,196,195,195,192,209,191,188
10H2,215,213,213,659,240,210,261
10CH4,197,195,195,670,nan,192,188
deltaH2,18.7,18.7,18.7,467,31.1,18.4,73.0


### 1.2 H2 loss
Hydrogen loss happens through two main processes. The largest loss is through soil sink. Remaining hydrogen is lost through reactions with OH as it ascends through the atmosphere. 

#### H2 dry deposition [Tg/yr]

The models diagnose soil sink based on their own schemes. Note that the dry deposition do not affect the atmospheric composition in the concentration driven runs, as the surface concentration of H2 are fixed in these runs.

In [6]:
file = 'H2_drydep.txt'
df_h2_drydep = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_drydep

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,59.5,73.0,52.0,54.6,58.4,44.3,56.6
10H2,65.5,80.3,57.1,204,67.8,48.7,79.2
10CH4,59.5,73.0,52.0,207,0.00,44.3,56.6


As drydeposition is uncertain, we can replace the models diagnosed drydeposition by a given value. In the concentration driven run, the drydeposition scheme do not impact the atmospheric composition. For emission driven runs, we also do adjust the drydeposition. Note that we do not use the emission numbers from the emission driven runs directly, the fluxes is estimated based on burden and lifetimes.

In [7]:
if(adjust_drydep):
    print('NB drydep adjusted')
    
    #Adjust by the relative adjustment in the control simulations
    adjust = drydep/df_h2_drydep.loc['CTRL']
    df_h2_drydep = df_h2_drydep*adjust
    
    print(df_h2_drydep)

#### H2 atmospheric loss [Tg/yr]

In [8]:
file = 'H2_atm_loss.txt'
df_h2_atmloss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmloss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,28.4,29.0,22.5,22.3,30.1,26.8,21.7
10H2,31.0,31.7,24.5,70.2,34.4,29.2,29.7
10CH4,27.6,28.2,22.0,69.5,0.00,26.0,21.2


#### H2 total loss [Tg/yr]:

In [9]:
df_h2_loss = df_h2_atmloss + df_h2_drydep
df_h2_loss


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,87.9,102,74.4,77.0,88.4,71.1,78.3
10H2,96.4,112,81.7,274,102,77.9,109
10CH4,87.2,101,73.9,276,0.00,70.3,77.8


### 1.3 H2 production

#### H2 atm. prod [Tg/yr]

In [10]:
file = 'H2_atm_prod.txt'
df_h2_atmprod = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_h2_atmprod

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,56.3,33.9,47.1,45.1,56.2,48.4,45.0
10H2,56.2,33.8,47.0,43.5,56.1,48.3,44.8
10CH4,59.2,36.4,49.4,45.7,0.00,50.8,47.1


### 1.4 Estimated H2 emissions (Total loss = Total prod + emission)

In the runs with fixed surface boundary conditions for H2, there are two unknowns; emissions and soil sink. The models do calculate the soil sink based on their own dry deposition schemes, but it does not influence the H2 in the atmosphere.  
Emission driven runs, are driven by emission estimates and use the dry deposition scheme to calculate the concentration at the surface. At steady state, the total production and total loss must balance:

In [11]:
df_h2_estemis = df_h2_atmloss + df_h2_drydep - df_h2_atmprod
df_h2_estemis

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,31.6,68.1,27.3,31.9,32.2,22.7,33.3
10H2,40.2,78.1,34.6,230,46.1,29.6,64.1
10CH4,28.0,64.8,24.6,230,0.00,19.5,30.7


In [12]:
df_h2_estemis.loc['10H2']-df_h2_estemis.loc['CTRL']

OSLOCTM3       8.63
WACCM6-2deg    10.0
INCA           7.29
GFDL-emi        199
OSLOCTM3-emi   13.9
UKCA           6.95
GFDL_nudge     30.8
dtype: float64

For the emission driven runs, the emissions calculated based on loss terms are similar to the emissions used in these runs.

### 1.5 H2 surface concentration [ppb]

In [13]:
file = 'H2_surfconc.txt'
df_h2_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_h2_surfconc.loc['10H2']-df_h2_surfconc.loc['CTRL']
delta.name = 'deltaH2'
df_h2_surfconc = df_h2_surfconc.append(delta)
df_h2_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,532,532,532,542,567,537,533
10H2,585,585,585,"1,912",656,590,746
deltaH2,53.2,53.2,53.2,"1,370",89.1,53.5,213


### 1.6 H2 lifetime [yr]
We calculate the lifetime as burden divided by loss.

#### H2 total lifetime [yr]

In [14]:
df_h2_lifetime = df_h2_burden.drop('deltaH2')/df_h2_loss
df_h2_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,2.23,1.91,2.61,2.50,2.36,2.69,2.40
10H2,2.23,1.91,2.61,2.41,2.35,2.69,2.40
10CH4,2.26,1.93,2.64,2.43,nan,2.73,2.42


#### H2 atmospheric lifetime [yr]
The atmospheric lifetime is the burden divided by only the atmospheric loss. This is the lifetime of the fraction of hydrogen which is not dry deposited.

In [15]:
df_h2_atm_lifetime = df_h2_burden.drop('deltaH2')/df_h2_atmloss
df_h2_atm_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,6.91,6.70,8.66,8.62,6.95,7.15,8.66
10H2,6.94,6.73,8.69,9.39,6.98,7.18,8.80
10CH4,7.12,6.93,8.88,9.64,nan,7.38,8.90


#### H2 soil sink lifetime [yr]

In [16]:
df_h2_soil_sink_lifetime = df_h2_burden.drop('deltaH2')/df_h2_drydep
df_h2_soil_sink_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,3.30,2.67,3.74,3.52,3.58,4.32,3.32
10H2,3.28,2.66,3.73,3.23,3.54,4.30,3.30
10CH4,3.31,2.67,3.75,3.24,nan,4.33,3.33


### 1.7 H2 flux  [Tg/yr]

The hydrogen flux is calculated as the burden divided by the total hydrogen lifetime. 

The hydrogen lifetime is calculated as burden divided by the total loss (soil sink + atm.loss). 

In stedy state, this is equal to the total loss (se check below)

The difference in calculated flux in the perturbed and control run is calculated. These numbers include feedbacks. 

For the GWP calculations, the radiative forcing in the steady state simulations are divided by these flux numbers.

In [17]:
df_h2_flux = df_h2_burden.drop('deltaH2')/df_h2_lifetime
#Add delta flux 10H2:
delta = df_h2_flux.loc['10H2']-df_h2_flux.loc['CTRL']
delta.name = 'deltaH2'
df_h2_flux = df_h2_flux.append(delta)
#Add delta flux 10CH4:
delta = df_h2_flux.loc['10CH4']-df_h2_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_h2_flux = df_h2_flux.append(delta)
df_h2_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,87.9,102,74.4,77.0,88.4,71.1,78.3
10H2,96.4,112,81.7,274,102,77.9,109
10CH4,87.2,101,73.9,276,nan,70.3,77.8
deltaH2,8.54,9.93,7.23,197,13.8,6.87,30.5
deltaCH4,-0.75,-0.86,-0.50,199,nan,-0.75,-0.52


In the methane run, the hydrogen surface concentration is kept fixed. Enhancing methane would influence H2. The hydrogen concentration would have increased, but since we run with fixed concentration, there is a negative flux to compensate. So the increased flux in H2 due to methane is -1*deltaCH4.

In [18]:
#Check
df_h2_loss

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,87.9,102,74.4,77.0,88.4,71.1,78.3
10H2,96.4,112,81.7,274,102,77.9,109
10CH4,87.2,101,73.9,276,0.00,70.3,77.8


## 2. Methane results

### 2.1 CH4 burden [Tg]

In [19]:
file = 'CH4_burden.txt'
df_ch4_burden = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_burden.loc['10CH4']-df_ch4_burden.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_burden = df_ch4_burden.append(delta)
df_ch4_burden

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,"4,995","5,003","5,003","4,955","4,995","4,939","4,876"
10H2,"4,995","5,003","5,003","4,954","4,995","4,939","4,876"
10CH4,"5,496","5,505","5,504","5,495",nan,"5,434","5,364"
deltaCH4,501,502,501,541,nan,495,488


### 2.2 CH4 atmospheric loss [Tg/yr]

This is atmospheric loss due to OH.

In [20]:
file = 'CH4_loss.txt'
df_ch4_loss = pd.read_csv(path + file, sep=';',index_col=0,header=0)
df_ch4_loss

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,683,727,593,540,681,624,528
10H2,680,724,590,499,677,621,520
10CH4,728,772,635,538,nan,664,564


### 2.3 CH4 surface concentration [ppb]

In [21]:
file = 'CH4_surfconc.txt'
df_ch4_surfconc = pd.read_csv(path + file, sep=';',index_col=0,header=0)
delta = df_ch4_surfconc.loc['10CH4']-df_ch4_surfconc.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_surfconc = df_ch4_surfconc.append(delta)
df_ch4_surfconc

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,"1,813","1,808","1,813","1,832","1,813","1,806","1,816"
10CH4,"1,994","1,989","1,994","2,032",nan,"1,986","1,998"
deltaCH4,181,181,181,200,nan,181,182


### 2.4 CH4 lifetime [yr]

#### Lifetime due to OH [yr]

In [22]:
df_ch4_lifetime = df_ch4_burden.drop('deltaCH4')/df_ch4_loss
df_ch4_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,7.31,6.88,8.44,9.17,7.33,7.92,9.24
10H2,7.34,6.91,8.47,9.93,7.38,7.95,9.38
10CH4,7.55,7.13,8.67,10.2,nan,8.19,9.51


#### Total CH4 lifetime [yr]

Inverse lifetimes (mean loss frequencies) are additive [Forster et al.,2007; Prather, 2007]. Added lifetime due to stratospheric chemistry in addion to OH and soil sink.

In [23]:
df_ch4_tot_lifetime = 1.0/(1.0/df_ch4_lifetime + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,6.80,6.42,7.76,8.37,6.81,7.31,8.43
10H2,6.82,6.45,7.79,9.00,6.85,7.34,8.55
10CH4,7.00,6.64,7.95,9.23,nan,7.54,8.66


In [24]:
#Double the tau_strat, to account for stratospheric chemistry not included in UKCA.
df_ch4_tot_lifetime['UKCA'] = 1.0/(1.0/df_ch4_lifetime + 1.0/tau_strat + 1.0/tau_strat + 1.0/tau_soil)
df_ch4_tot_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,6.80,6.42,7.76,8.37,6.81,6.61,8.43
10H2,6.82,6.45,7.79,9.00,6.85,6.63,8.55
10CH4,7.00,6.64,7.95,9.23,nan,6.80,8.66


#### Other methane loss term, dervided based on the lifetime and burden.

In [25]:
#Soil loss:
df_ch4_loss_soil = df_ch4_burden.drop('deltaCH4')/tau_soil
df_ch4_loss_soil

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,31.2,31.3,31.3,31.0,31.2,30.9,30.5
10H2,31.2,31.3,31.3,31.0,31.2,30.9,30.5
10CH4,34.3,34.4,34.4,34.3,nan,34.0,33.5


In [26]:
#Strat chemistry loss (other than OH)
df_ch4_loss_other_strat = df_ch4_burden.drop('deltaCH4')/(tau_strat)
df_ch4_loss_other_strat

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,20.8,20.8,20.8,20.6,20.8,20.6,20.3
10H2,20.8,20.8,20.8,20.6,20.8,20.6,20.3
10CH4,22.9,22.9,22.9,22.9,nan,22.6,22.4


### 2.5 CH4 flux  [Tg/yr]

The methane flux is calculated as the burden divided by the total methane lifetime.

The difference in calculated flux in the perturbed and control run is calculated. These numbers include feedbacks.

In [27]:
df_ch4_flux = df_ch4_burden.drop('deltaCH4')/df_ch4_tot_lifetime
#Add delta CH4 flux 10H2
delta = df_ch4_flux.loc['10H2']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaH2'
df_ch4_flux = df_ch4_flux.append(delta)
#Add delta CH4 flux 10CH4
delta = df_ch4_flux.loc['10CH4']-df_ch4_flux.loc['CTRL']
delta.name = 'deltaCH4'
df_ch4_flux = df_ch4_flux.append(delta)


#Add same flux in OsloCTM3-emi as in OsloCTM3
df_ch4_flux['OSLOCTM3-emi'].loc['deltaCH4']= df_ch4_flux['OSLOCTM3'].loc['deltaCH4']
df_ch4_flux


,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,735,779,645,592,733,747,578
10H2,732,776,643,550,729,745,570
10CH4,785,829,692,595,nan,799,620
deltaH2,-2.67,-3.28,-2.12,-41.7,-4.39,-2.62,-7.96
deltaCH4,50.3,50.3,47.3,3.10,50.3,51.8,41.4


## 3. Ozone burden and RF

### 3.1 Change in tropospheric ozone (DU)

The tropopause definition is the model layer in the control simulation where 150 ppbv ozone are reached. 

Keep in mind that:
for GFDL-emi 10H2 is H2 perturbation (200 Tg/yr) and 10CH4 is H2+CH4 pert. And that for OsloCTM-emi, the same sensitivity for methane perturbation is used as for OsloCTM concentration driven.

In [28]:
file = 'ozone_du_trop.txt'
df_ozone_du_trop = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_trop

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
10H2,0.04,0.03,0.03,0.70,0.06,0.03,0.12
10CH4,0.83,0.88,0.53,1.57,nan,0.70,0.66


In [29]:
#For OsloCTM-emi, use the methane results from the concentration driven methane experiment.
df_ozone_du_trop['OSLOCTM3-emi'].loc['10CH4'] = df_ozone_du_trop['OSLOCTM3'].loc['10CH4'] 
df_ozone_du_trop

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
10H2,0.04,0.03,0.03,0.70,0.06,0.03,0.12
10CH4,0.83,0.88,0.53,1.57,0.83,0.70,0.66


### 3.2 Change in stratospheric ozone (DU)

In [30]:
file = 'ozone_du_strat.txt'
df_ozone_du_strat = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_du_strat

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
10H2,0.01,-0.01,-0.04,-0.38,0.01,nan,-0.07
10CH4,0.83,0.80,0.25,1.03,nan,nan,0.78


In [31]:
#For OsloCTM-emi, use the methane results from the concentration driven methane experiment.
df_ozone_du_strat['OSLOCTM3-emi'].loc['10CH4'] = df_ozone_du_strat['OSLOCTM3'].loc['10CH4'] 
df_ozone_du_strat

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
10H2,0.01,-0.01,-0.04,-0.38,0.01,nan,-0.07
10CH4,0.83,0.80,0.25,1.03,0.83,nan,0.78


### 3.5 Ozone RF

Ozone RF is calculated using a radiative kernel (Skeie et al 2020) and the modelled changes in ozone. 

Keep in mind that:
for GFDL-emi 10H2 RF is forcing calculated in the H2 perturbation and 10CH4 the forcing calculated by the H2+CH4.

In [32]:
file = 'ozone_rf.txt'
df_ozone_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_ozone_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
10H2,1.86,1.75,1.06,31.0,3.06,1.14,6.40
10CH4,41.0,42.7,22.1,76.5,nan,24.3,33.4


In [33]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_ozone_rf['OSLOCTM3-emi'].loc['10CH4'] = df_ozone_rf['OSLOCTM3'].loc['10CH4'] 
df_ozone_rf

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
10H2,1.86,1.75,1.06,31.0,3.06,1.14,6.40
10CH4,41.0,42.7,22.1,76.5,41.0,24.3,33.4


## 4. Stratospheric H2O RF [mW m-2]

Stratospheric H2O RF calculated offline.

In [34]:
file = 'H2O_rf.txt'
df_h2o_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_h2o_rf

,OSLOCTM3,WACCM6-2deg,INCA,OSLOCTM3-emi,GFDL_nudge,GFDL-emi
Scenario,,,,,,
10H2,1.48,0.51,0.82,2.43,4.92,27.2
10CH4,10.1,3.76,6.62,nan,20.0,49.3


In [35]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_h2o_rf['OSLOCTM3-emi'].loc['10CH4'] = df_h2o_rf['OSLOCTM3'].loc['10CH4'] 
df_h2o_rf

,OSLOCTM3,WACCM6-2deg,INCA,OSLOCTM3-emi,GFDL_nudge,GFDL-emi
Scenario,,,,,,
10H2,1.48,0.51,0.82,2.43,4.92,27.2
10CH4,10.1,3.76,6.62,10.1,20.0,49.3


## 5. Aerosol RF [mW m-2]

In [36]:
file = 'aerosol_rf.txt'
df_aerosol_rf = pd.read_csv(path+file, sep=';',index_col=0,header=0)
df_aerosol_rf

,OSLOCTM3,GFDL-emi,GFDL_nudge,OSLOCTM3-emi,INCA
Scenario,,,,,
10H2,-0.28,-20.0,-0.12,-0.39,0.22
10CH4,-1.11,-14.4,-0.37,nan,2.62


In [37]:
#In OsloCTM-emi use the same RF in the methane pertubation as in OsloCTM3
df_aerosol_rf['OSLOCTM3-emi'].loc['10CH4'] = df_aerosol_rf['OSLOCTM3'].loc['10CH4'] 
df_aerosol_rf

,OSLOCTM3,GFDL-emi,GFDL_nudge,OSLOCTM3-emi,INCA
Scenario,,,,,
10H2,-0.28,-20.0,-0.12,-0.39,0.22
10CH4,-1.11,-14.4,-0.37,-1.11,2.62



# Part II: GWP calculations:

The pulse integrated to infinity of the effects of a short lived climate forcer is equal to the change respones of its effects at steady state multiplied by the steady state lifetime of the short lived forcer(:cite:p:`Prather2002a` and :cite:p:`Prather2007a`). 

Prather 2002: prove that: (a) the steadystate pattern of impacts caused by specified emissions, multiplied by (b) the steady-state lifetime of the source gas for that emission pattern, is exactly equal to (c) the integral of all impacts - independent of the number and atmospheric residence times of secondary impacts. Therefore, the AGWP for hydrogen is identical whether calculating by integrating a pulse or by using the steady state changes per flux, given that the perturbation reaches steady state before 100 years. The longest-lived chemical mode here is keyed to methane, which has an e-folding lifetime of about 12 years. Our perturbed experiments are run to steate-state.


### Change in H2 surface conc. caused by 1 Tg H2/yr [ppb yr Tg-1]

This is not used for the GWP calculation. Only for the per flux table and for the feedback factor calulations.

In [38]:
df_surf_h2_per_h2_flux = df_h2_surfconc.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_surf_h2_per_h2_flux.name = 'surf_h2_per_h2_flux'
df_surf_h2_per_h2_flux

OSLOCTM3       6.23
WACCM6-2deg    5.36
INCA           7.36
GFDL-emi       6.96
OSLOCTM3-emi   6.48
UKCA           7.80
GFDL_nudge     6.98
Name: surf_h2_per_h2_flux, dtype: float64

### Change in CH4 flux caused by 1 TgH2 /yr (includes H2 feedback) [Tg CH4/Tg H2]:

The ch4_flux is multiplied by -1 (see above).

In [39]:
df_ch4_flux_per_h2_flux = -1.0*df_ch4_flux.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_ch4_flux_per_h2_flux.name = 'ch4_flux_per_h2_flux'
df_ch4_flux_per_h2_flux

OSLOCTM3       0.31
WACCM6-2deg    0.33
INCA           0.29
GFDL-emi       0.21
OSLOCTM3-emi   0.32
UKCA           0.38
GFDL_nudge     0.26
Name: ch4_flux_per_h2_flux, dtype: float64

### Change in CH4 surface conc. caused by 1 Tg/yr CH4 [ppb yr/Tg CH4]

In [40]:
df_surf_ch4_per_ch4_flux =  df_ch4_surfconc.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_surf_ch4_per_ch4_flux.name = 'surf_ch4_per_ch4_flux'
df_surf_ch4_per_ch4_flux

OSLOCTM3       3.61
WACCM6-2deg    3.60
INCA           3.83
GFDL-emi       64.4
OSLOCTM3-emi    nan
UKCA           3.49
GFDL_nudge     4.39
Name: surf_ch4_per_ch4_flux, dtype: float64

OsloCTM3-emi set equal to OsloCTM3 concentration driven.

In [41]:
df_surf_ch4_per_ch4_flux['OSLOCTM3-emi'] = df_surf_ch4_per_ch4_flux['OSLOCTM3']
df_surf_ch4_per_ch4_flux

OSLOCTM3       3.61
WACCM6-2deg    3.60
INCA           3.83
GFDL-emi       64.4
OSLOCTM3-emi   3.61
UKCA           3.49
GFDL_nudge     4.39
Name: surf_ch4_per_ch4_flux, dtype: float64

### Change in CH4 surface concentration per emission H2 [ppb yr /Tg H2]

In [42]:
df_surf_ch4_per_h2_flux = df_surf_ch4_per_ch4_flux*df_ch4_flux_per_h2_flux
df_surf_ch4_per_h2_flux.name = 'surf_ch4_per_h2_flux'

In [43]:
#Add GFDL:
#Increase in surface concentration CH4:
#1808 to 2005 ppbv (REF Paolot)
print(df_ch4_surfconc['GFDL-emi'])
print(2005.-1808.)
#Might be an issue with with dry or ambient. The difference is similar though: 200 vs 197.
df_surf_ch4_per_h2_flux['GFDL-emi']= (2005.-1808.)/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_surf_ch4_per_h2_flux

Scenario
CTRL       1,832
10CH4      2,032
deltaCH4     200
Name: GFDL-emi, dtype: float64
197.0


OSLOCTM3       1.13
WACCM6-2deg    1.19
INCA           1.13
GFDL-emi       1.00
OSLOCTM3-emi   1.15
UKCA           1.33
GFDL_nudge     1.14
Name: surf_ch4_per_h2_flux, dtype: float64

### Change in H2 flux caused by 1 TgCH4/yr [Tg H2/Tg CH4]

We multiply by -1 (see above)

In [44]:
df_h2_flux_per_ch4_flux = -1.0*df_h2_flux.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_h2_flux_per_ch4_flux.name = 'h2_flux_per_ch4_flux'
df_h2_flux_per_ch4_flux

OSLOCTM3        0.01
WACCM6-2deg     0.02
INCA            0.01
GFDL-emi       -64.3
OSLOCTM3-emi     nan
UKCA            0.01
GFDL_nudge      0.01
Name: h2_flux_per_ch4_flux, dtype: float64

In [45]:
df_h2_flux_per_ch4_flux['OSLOCTM3-emi'] = df_h2_flux_per_ch4_flux['OSLOCTM3']
df_h2_flux_per_ch4_flux

OSLOCTM3        0.01
WACCM6-2deg     0.02
INCA            0.01
GFDL-emi       -64.3
OSLOCTM3-emi    0.01
UKCA            0.01
GFDL_nudge      0.01
Name: h2_flux_per_ch4_flux, dtype: float64

### HYDROGEN AGWP100 CH4 [mW m-2 yr Tg-1]

agwp_ch4 = RF per flux H2

In [46]:
df_h2_agwp_ch4 = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_h2_agwp_ch4.name = 'h2_agwp_ch4'


#agwp_ch4 = RF per flux H2 (For the per flux table)
df_ch4_rf_per_h2_flux = df_surf_ch4_per_h2_flux*spec_rf_ch4
df_ch4_rf_per_h2_flux.name = 'ch4_rf_per_h2_flux'

df_ch4_rf_per_h2_flux

OSLOCTM3       0.43
WACCM6-2deg    0.46
INCA           0.43
GFDL-emi       0.39
OSLOCTM3-emi   0.44
UKCA           0.51
GFDL_nudge     0.44
Name: ch4_rf_per_h2_flux, dtype: float64

### Initialize H2 GWP table

In [47]:
antmod = len(df_h2_agwp_ch4.index)
df_h2_gwp = pd.DataFrame(np.empty([5,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','O3 CH4ind','strat H2O CH4ind'])

### Add methane GWP

In [48]:
df_h2_gwp.loc['CH4'] = df_h2_agwp_ch4/agwp100_CO2

### HYDROGEN AGWP100 strat H2O [mW m-2 yr Tg-1]

In [49]:
df_h2_agwp_h2o = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']

#GFDL emis, that include methane induced changes. 
df_h2_agwp_h2o['GFDL-emi'] = df_h2o_rf['GFDL-emi'].loc['10CH4']/df_h2_flux['GFDL-emi'].loc['deltaH2']
df_h2_agwp_h2o.name = 'h2_agwp_h2o'


In [50]:
#Add to the flux table
df_h2o_rf_per_h2_flux = df_h2o_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2o_rf_per_h2_flux.name= 'h2o_rf_per_h2_flux'

#For GFDL-emi, this per h2 flux include methane induced changes.
df_h2o_rf_per_h2_flux['GFDL-emi'] = df_h2o_rf['GFDL-emi'].loc['10CH4']/df_h2_flux['GFDL-emi'].loc['deltaH2']

#Strat H2O RF per methane flux 
df_h2o_rf_per_ch4_flux = df_h2o_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_h2o_rf_per_ch4_flux.name = 'h2o_rf_per_ch4_flux'


In [51]:
df_h2o_rf_per_ch4_flux['OSLOCTM3-emi'] = df_h2o_rf_per_ch4_flux['OSLOCTM3']
#df_h2o_rf_per_ch4_flux

### Add stratospheric H2O GWP

In [52]:
df_h2_gwp.loc['strat H2O'] = df_h2_agwp_h2o/agwp100_CO2

### HYDROGEN AGWP100 O3 [mW m-2 yr Tg-1]

In [53]:
df_h2_agwp_o3 = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_o3.name = 'h2_agwp_o3'

#For GFDL-emi include the methane induced part.
df_h2_agwp_o3['GFDL-emi'] = df_ozone_rf['GFDL-emi'].loc['10CH4']/df_h2_flux['GFDL-emi'].loc['deltaH2']


In [54]:
#Similar, but use only the H2 Ozone RF for GFDL. To be used in the table:
df_ozone_rf_per_h2_flux = df_ozone_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_ozone_rf_per_h2_flux.name= 'ozone_rf_per_h2_flux'

#For GFDL-emi, this include the methane change
df_ozone_rf_per_h2_flux['GFDL-emi'] = df_ozone_rf['GFDL-emi'].loc['10CH4']/df_h2_flux['GFDL-emi'].loc['deltaH2']

In [55]:
#Ozone RF per methane flux (move to the methane part?)
df_ozone_rf_per_ch4_flux = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ozone_rf_per_ch4_flux.name = 'ozone_rf_per_ch4_flux'

In [56]:
df_ozone_rf_per_ch4_flux['OSLOCTM3-emi'] = df_ozone_rf_per_ch4_flux['OSLOCTM3']
df_ozone_rf_per_ch4_flux

OSLOCTM3       0.82
WACCM6-2deg    0.85
INCA           0.47
GFDL-emi       24.7
OSLOCTM3-emi   0.82
UKCA           0.47
GFDL_nudge     0.81
Name: ozone_rf_per_ch4_flux, dtype: float64

### Add Ozone GWP

In [57]:
df_h2_gwp.loc['O3'] = df_h2_agwp_o3/agwp100_CO2

### For the per flux table

In [58]:
df_trop_du_ozone_per_ch4_flux = df_ozone_du_trop.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_trop_du_ozone_per_ch4_flux.name = 'trop_du_ozone_per_ch4_flux'

df_strat_du_ozone_per_ch4_flux = df_ozone_du_strat.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_strat_du_ozone_per_ch4_flux.name = 'strat_du_ozone_per_ch4_flux'

df_trop_du_ozone_per_h2_flux = df_ozone_du_trop.loc['10H2']/df_h2_flux.loc['deltaH2']
df_trop_du_ozone_per_h2_flux.name = 'trop_du_ozone_per_h2_flux'


#GFDL emis, that include methane induced changes. 
df_trop_du_ozone_per_h2_flux['GFDL-emi'] = df_ozone_du_trop['GFDL-emi'].loc['10CH4']/df_h2_flux['GFDL-emi'].loc['deltaH2']

df_strat_du_ozone_per_h2_flux = df_ozone_du_strat.loc['10H2']/df_h2_flux.loc['deltaH2']
df_strat_du_ozone_per_h2_flux.name = 'strat_du_ozone_per_h2_flux'

#GFDL emis, that include methane induced changes. 
df_strat_du_ozone_per_h2_flux['GFDL-emi'] = df_ozone_du_strat['GFDL-emi'].loc['10CH4']/df_h2_flux['GFDL-emi'].loc['deltaH2']

In [59]:
df_trop_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_trop_du_ozone_per_ch4_flux['OSLOCTM3']
df_strat_du_ozone_per_ch4_flux['OSLOCTM3-emi']=df_strat_du_ozone_per_ch4_flux['OSLOCTM3']

### HYDROGEN AGWP100 aerosol [mW m-2 yr Tg-1]

In [60]:
df_h2_agwp_aerosol = df_aerosol_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_h2_agwp_aerosol.name = 'h2_agwp_aerosol'
df_h2_agwp_aerosol
#NBNB GFDL-emi include methane induced.

GFDL-emi       -0.10
GFDL_nudge     -0.00
INCA            0.03
OSLOCTM3       -0.03
OSLOCTM3-emi   -0.03
UKCA             nan
WACCM6-2deg      nan
Name: h2_agwp_aerosol, dtype: float64

In [61]:
df_h2_gwp.loc['aerosol'] = df_h2_agwp_aerosol/agwp100_CO2 

In [62]:
#Add to the flux table
df_aerosol_rf_per_h2_flux = df_aerosol_rf.loc['10H2']/df_h2_flux.loc['deltaH2']
df_aerosol_rf_per_h2_flux.name= 'aerosol_rf_per_h2_flux'

df_aerosol_rf_per_h2_flux['GFDL-emi']=df_aerosol_rf['GFDL-emi'].loc['10CH4']/df_h2_flux['GFDL-emi'].loc['deltaH2']


df_aerosol_rf_per_ch4_flux = df_aerosol_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_aerosol_rf_per_ch4_flux.name = 'aerosol_rf_per_ch4_flux'


In [63]:
df_aerosol_rf_per_ch4_flux['OSLOCTM3-emi']=df_aerosol_rf_per_ch4_flux['OSLOCTM3']

## Methane induced GWP:

### HYDROGEN AGWP100 methane induced O3 [mW m-2 yr Tg-1]

It does not matter here if we use surface concentration, burden or tropospheric concentration.

In [64]:
df_ch4_surfconc['OSLOCTM3-emi'].loc['deltaCH4']=df_ch4_surfconc['OSLOCTM3'].loc['deltaCH4']

In [65]:
#Wm-2/ppbCH4*ppbCH4/TgH2yr-1 -> Wm-2/TgH2yr-1
df_h2_agwp_ch4ind_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_o3.name = 'h2_agwp_ch4ind_o3'
df_h2_agwp_ch4ind_o3['GFDL-emi']=np.nan

### Add methane induced O3 GWP

In [66]:
df_h2_gwp.loc['O3 CH4ind'] = df_h2_agwp_ch4ind_o3/agwp100_CO2
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,2.44,1.97,1.64,4.34,2.49,1.85,2.34
CH4,4.85,5.11,4.85,4.30,4.96,5.73,4.92
strat H2O,1.94,0.58,1.27,2.80,1.97,nan,1.80
O3 CH4ind,2.85,3.13,1.53,nan,2.91,2.00,2.35
strat H2O CH4ind,nan,nan,nan,nan,nan,nan,nan
aerosol,-0.37,nan,0.34,-1.14,-0.32,nan,-0.05


### HYDROGEN AGWP100 methane induced strat H2O [mW m-2 yr Tg-1]

In [67]:
df_h2_agwp_ch4ind_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_h2o.name = 'h2_agwp_ch4ind_h2o'
df_h2_agwp_ch4ind_h2o['GFDL-emi']=np.nan

### Add methane induced strat H2O GWP

In [68]:
df_h2_gwp.loc['strat H2O CH4ind'] = df_h2_agwp_ch4ind_h2o/agwp100_CO2
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,2.44,1.97,1.64,4.34,2.49,1.85,2.34
CH4,4.85,5.11,4.85,4.30,4.96,5.73,4.92
strat H2O,1.94,0.58,1.27,2.80,1.97,nan,1.80
O3 CH4ind,2.85,3.13,1.53,nan,2.91,2.00,2.35
strat H2O CH4ind,0.70,0.28,0.46,nan,0.72,nan,1.41
aerosol,-0.37,nan,0.34,-1.14,-0.32,nan,-0.05


### HYDROGEN AGWP100 methane induced aerosols [mW m-2 yr Tg-1]

In [69]:
df_h2_agwp_ch4ind_aerosol = df_aerosol_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_h2_flux
df_h2_agwp_ch4ind_aerosol.name = 'h2_agwp_ch4ind_aerosols'
df_h2_agwp_ch4ind_aerosol['GFDL-emi']=np.nan

### Hydrogen GWP including aerosols

In [70]:
df_h2_gwp.loc['aerosol CH4ind'] = df_h2_agwp_ch4ind_aerosol/agwp100_CO2
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,2.44,1.97,1.64,4.34,2.49,1.85,2.34
CH4,4.85,5.11,4.85,4.30,4.96,5.73,4.92
strat H2O,1.94,0.58,1.27,2.80,1.97,nan,1.80
O3 CH4ind,2.85,3.13,1.53,nan,2.91,2.00,2.35
strat H2O CH4ind,0.70,0.28,0.46,nan,0.72,nan,1.41
aerosol,-0.37,nan,0.34,-1.14,-0.32,nan,-0.05
aerosol CH4ind,-0.08,nan,0.18,nan,-0.08,nan,-0.03


In [71]:
#Not include the aerosol GWP in the main table. Drop them here
df_h2_gwp = df_h2_gwp.drop(['aerosol','aerosol CH4ind'])
df_h2_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,2.44,1.97,1.64,4.34,2.49,1.85,2.34
CH4,4.85,5.11,4.85,4.30,4.96,5.73,4.92
strat H2O,1.94,0.58,1.27,2.80,1.97,nan,1.80
O3 CH4ind,2.85,3.13,1.53,nan,2.91,2.00,2.35
strat H2O CH4ind,0.70,0.28,0.46,nan,0.72,nan,1.41


# Methane GWP

Initialize CH4 GWP

In [72]:
antmod = len(df_h2_agwp_ch4.index)
df_ch4_gwp = pd.DataFrame(np.empty([4,antmod])*np.nan,columns=df_h2_agwp_ch4.index,
                         index=['O3','CH4','strat H2O','H2'])
#df_ch4_gwp

### Methane AGWP100 O3 [mW m-2 yr Tg-1]

In [73]:
df_ch4_agwp_o3 = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_o3.name = 'ch4_agwp_o3'
df_ch4_agwp_o3['GFDL-emi']=np.nan
df_ch4_agwp_o3

OSLOCTM3       0.82
WACCM6-2deg    0.85
INCA           0.47
GFDL-emi        nan
OSLOCTM3-emi   0.82
UKCA           0.47
GFDL_nudge     0.81
Name: ch4_agwp_o3, dtype: float64

In [74]:
test = df_ozone_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
#test

In [75]:
test2 = df_ozone_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
#test2 is equal test

### Add ozone GWP

In [76]:
df_ch4_gwp.loc['O3'] =df_ch4_agwp_o3/agwp100_CO2 

### Methane AGWP100 Methane [mW m-2 yr Tg-1]

In [77]:
df_ch4_agwp =df_surf_ch4_per_ch4_flux*spec_rf_ch4
df_ch4_agwp.name = 'ch4_agwp'
df_ch4_agwp['GFDL-emi']=np.nan

### Add methane GWP

In [78]:
#Add Methane GWP:
df_ch4_gwp.loc['CH4'] =df_ch4_agwp/agwp100_CO2 

### Methane AGWP100 strat H2O [mW m-2 yr Tg-1]

In [79]:
df_ch4_agwp_h2o = df_h2o_rf.loc['10CH4']/df_ch4_surfconc.loc['deltaCH4']*df_surf_ch4_per_ch4_flux
df_ch4_agwp_h2o.name = 'ch4_agwp_h2o'
df_ch4_agwp_h2o['GFDL-emi']=np.nan

### Add Strat H2O GWP:

In [80]:
df_ch4_gwp.loc['strat H2O'] = df_ch4_agwp_h2o/agwp100_CO2

### Methane AGWP100 aerosols [mW m-2 yr Tg-1]

In [81]:
df_ch4_agwp_aerosol = df_aerosol_rf.loc['10CH4']/df_ch4_flux.loc['deltaCH4']
df_ch4_agwp_aerosol.name = 'ch4_agwp_aerosol'
df_ch4_agwp_aerosol['GFDL-emi']=np.nan

### Add Aerosol GWP

In [82]:
df_ch4_gwp.loc['aerosol'] = df_ch4_agwp_aerosol/agwp100_CO2
df_ch4_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,9.12,9.48,5.21,nan,9.12,5.23,9.02
CH4,15.5,15.5,16.5,nan,15.5,15.0,18.9
strat H2O,2.25,0.84,1.57,nan,2.25,nan,5.40
H2,nan,nan,nan,nan,nan,nan,nan
aerosol,-0.25,nan,0.62,nan,-0.25,nan,-0.10


In [83]:
#And drop the aerosol GWP in the main table:
df_ch4_gwp = df_ch4_gwp.drop(['aerosol'])
df_ch4_gwp

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
O3,9.12,9.48,5.21,nan,9.12,5.23,9.02
CH4,15.5,15.5,16.5,nan,15.5,15.0,18.9
strat H2O,2.25,0.84,1.57,nan,2.25,nan,5.40
H2,nan,nan,nan,nan,nan,nan,nan


### Add GWP via H2

In [84]:
df_ch4_gwp.loc['H2'] = df_h2_flux_per_ch4_flux*df_h2_gwp.sum()
df_ch4_gwp['GFDL-emi'].loc['H2']=np.nan

# Part III: Main results and tables

## H2 GWP 100

In [85]:
model_dict = { 'OSLOCTM3':'OsloCTM',
               'WACCM6-2deg':'WACCM',
               'INCA':'INCA',
               'GFDL-emi':'GFDL-emi',
               'GFDL_nudge':'GFDL',
               'UKCA':'UKCA',
               'OSLOCTM3-emi':'OsloCTM-emi'}

sorted_array = ['GFDL','OsloCTM','INCA','UKCA','WACCM','GFDL-emi','OsloCTM-emi']
sorted_array_2 = ['GFDL','OsloCTM','INCA','UKCA','WACCM','GFDL-emi','OsloCTM-emi','Model mean']

In [86]:
df_h2_gwp = df_h2_gwp[sorted(df_h2_gwp.columns)]

df_h2_gwp.loc['total']=df_h2_gwp.sum()

df_h2_gwp_table = df_h2_gwp.copy()

df_h2_gwp_table.rename(model_dict,axis=1,inplace=True)
df_h2_gwp_table = df_h2_gwp_table[sorted_array]
df_h2_gwp_table.T


/home/ragnhibs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,O3,CH4,strat H2O,O3 CH4ind,strat H2O CH4ind,total
GFDL,2.34,4.92,1.80,2.35,1.41,12.8
OsloCTM,2.44,4.85,1.94,2.85,0.70,12.8
INCA,1.64,4.85,1.27,1.53,0.46,9.76
UKCA,1.85,5.73,nan,2.00,nan,9.59
WACCM,1.97,5.11,0.58,3.13,0.28,11.1
GFDL-emi,4.34,4.30,2.80,nan,nan,11.4
OsloCTM-emi,2.49,4.96,1.97,2.91,0.72,13.0


## CH4 GWP 100

In [87]:
df_ch4_gwp = df_ch4_gwp[sorted(df_ch4_gwp.columns)]
df_ch4_gwp.drop('GFDL-emi',axis=1).to_csv(outputpath + 'table_ch4_gwp.csv')
df_ch4_gwp.loc['total']=df_ch4_gwp.sum()
df_ch4_gwp.T

/home/ragnhibs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,O3,CH4,strat H2O,H2,total
GFDL-emi,nan,nan,nan,nan,0.00
GFDL_nudge,9.02,18.9,5.40,0.16,33.5
INCA,5.21,16.5,1.57,0.10,23.4
OSLOCTM3,9.12,15.5,2.25,0.19,27.1
OSLOCTM3-emi,9.12,15.5,2.25,0.19,27.1
UKCA,5.23,15.0,nan,0.14,20.4
WACCM6-2deg,9.48,15.5,0.84,0.19,26.0


## Table per flux H2

In [88]:
df_per_flux_h2 = pd.concat([df_h2_flux.loc['deltaH2'],
                            df_surf_h2_per_h2_flux,
                            df_surf_ch4_per_h2_flux,
                            df_ch4_flux_per_h2_flux,
                            df_ch4_rf_per_h2_flux,
                            df_trop_du_ozone_per_h2_flux*1000.,
                            df_strat_du_ozone_per_h2_flux*1000.,
                            df_ozone_rf_per_h2_flux,
                            df_h2o_rf_per_h2_flux,
                            df_aerosol_rf_per_h2_flux],axis=1, sort=False)

#NB in this the methane induced changes for GFDL-emi is included.

#Save to file:
df_per_flux_h2 = df_per_flux_h2.sort_index()
df_per_flux_h2.to_csv(outputpath + 'table_per_flux_h2.csv')

#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_h2_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
#Rename column names:
df_per_flux_h2.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_h2.rename(model_dict,axis=0,inplace=True)
df_per_flux_h2.loc['Model mean'] = df_per_flux_h2.drop(['GFDL-emi','OsloCTM-emi']).mean()
df_per_flux_h2['Flux H2 [Tg/yr]'].loc['Model mean']=np.nan
df_per_flux_h2=df_per_flux_h2.reindex(sorted_array_2)
df_per_flux_h2.drop('GFDL-emi')

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],Trop. ozone per flux [10$^{-3}$ DU yr/Tg],Strat. ozone per flux [10$^{-3}$ DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg],Aerosol RF per flux [mW m-2 yr/ Tg]
GFDL,30.5,6.98,1.14,0.26,0.44,4.09,-2.42,0.21,0.16,-0.00
OsloCTM,8.54,6.23,1.13,0.31,0.43,4.48,0.77,0.22,0.17,-0.03
INCA,7.23,7.36,1.13,0.29,0.43,3.93,-5.56,0.15,0.11,0.03
UKCA,6.87,7.80,1.33,0.38,0.51,4.63,nan,0.17,nan,nan
WACCM,9.93,5.36,1.19,0.33,0.46,3.31,-1.10,0.18,0.05,nan
OsloCTM-emi,13.8,6.48,1.15,0.32,0.44,4.61,0.75,0.22,0.18,-0.03
Model mean,nan,6.74,1.18,0.32,0.46,4.09,-2.08,0.18,0.13,-0.00


## Table per flux CH4

In [89]:
df_per_flux_ch4 = pd.concat([df_ch4_flux.loc['deltaCH4'],
                            df_surf_ch4_per_ch4_flux,
                            df_h2_flux_per_ch4_flux,
                            df_trop_du_ozone_per_ch4_flux*1000.,
                            df_strat_du_ozone_per_ch4_flux*1000.,
                            df_ozone_rf_per_ch4_flux,
                            df_h2o_rf_per_ch4_flux,
                            df_aerosol_rf_per_ch4_flux],axis=1,sort=False)



               
#Save to file:
df_per_flux_ch4 = df_per_flux_ch4.sort_index()
df_per_flux_ch4.to_csv(outputpath + 'table_per_flux_ch4.csv')

#Rename the columns:
columns_names={'deltaCH4':'Flux CH4 [Tg/yr]',
               'surf_ch4_per_ch4_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'h2_flux_per_ch4_flux':'Flux H2/Flux CH4 [Tg H2/Tg CH4]',
               'trop_du_ozone_per_ch4_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_ch4_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_ch4_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_ch4_flux':'Strat H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_ch4_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
               
#Rename column names:

df_per_flux_ch4.rename(columns=dict(columns_names),inplace=True) 

df_per_flux_ch4.rename(model_dict, inplace=True)

df_per_flux_ch4.loc['Model mean'] = df_per_flux_ch4.drop(['GFDL-emi','OsloCTM-emi']).mean()

df_per_flux_ch4 = df_per_flux_ch4.reindex(sorted_array_2)

df_per_flux_ch4.drop('GFDL-emi')          

,Flux CH4 [Tg/yr],Surf. conc. CH4 per flux [ppb yr/Tg],Flux H2/Flux CH4 [Tg H2/Tg CH4],Trop. ozone per flux [10$^{-3}$ DU yr/Tg],Strat. ozone per flux [10$^{-3}$ DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat H2O RF per flux [mW m-2 yr/ Tg],Aerosol RF per flux [mW m-2 yr/ Tg]
GFDL,41.4,4.39,0.01,16.0,18.8,0.81,0.48,-0.01
OsloCTM,50.3,3.61,0.01,16.5,16.4,0.82,0.20,-0.02
INCA,47.3,3.83,0.01,11.3,5.29,0.47,0.14,0.06
UKCA,51.8,3.49,0.01,13.6,nan,0.47,nan,nan
WACCM,50.3,3.60,0.02,17.4,15.9,0.85,0.07,nan
OsloCTM-emi,50.3,3.61,0.01,16.5,16.4,0.82,0.20,-0.02
Model mean,48.2,3.78,0.01,15.0,14.1,0.68,0.22,0.01


## Table per flux H2 (including changes in methane)

In [90]:
#Reread - to get the other heading.
#GFDL model alreade include methane induced changes.

df_per_flux_h2_combined = pd.read_csv(outputpath + 'table_per_flux_h2.csv',index_col=0)
df_per_flux_h2_combined.rename(columns=dict(columns_names),inplace=True) 

df_per_flux_h2_combined.rename(model_dict, inplace=True)

df_per_flux_h2_combined

,deltaH2,surf_h2_per_h2_flux,surf_ch4_per_h2_flux,ch4_flux_per_h2_flux,ch4_rf_per_h2_flux,trop_du_ozone_per_h2_flux,strat_du_ozone_per_h2_flux,ozone_rf_per_h2_flux,h2o_rf_per_h2_flux,aerosol_rf_per_h2_flux
GFDL-emi,197,6.96,1.00,0.21,0.39,7.97,5.23,0.39,0.25,-0.07
GFDL,30.5,6.98,1.14,0.26,0.44,4.09,-2.42,0.21,0.16,-0.00
INCA,7.23,7.36,1.13,0.29,0.43,3.93,-5.56,0.15,0.11,0.03
OsloCTM,8.54,6.23,1.13,0.31,0.43,4.48,0.77,0.22,0.17,-0.03
OsloCTM-emi,13.8,6.48,1.15,0.32,0.44,4.61,0.75,0.22,0.18,-0.03
UKCA,6.87,7.80,1.33,0.38,0.51,4.63,nan,0.17,nan,nan
WACCM,9.93,5.36,1.19,0.33,0.46,3.31,-1.10,0.18,0.05,nan


In [91]:
df_per_flux_ch4_add  = pd.read_csv(outputpath + 'table_per_flux_ch4.csv',index_col=0)
df_per_flux_ch4_add.rename(model_dict, inplace=True)

df_per_flux_ch4_add.loc['GFDL-emi'] = 0.0
df_per_flux_ch4_add

,deltaCH4,surf_ch4_per_ch4_flux,h2_flux_per_ch4_flux,trop_du_ozone_per_ch4_flux,strat_du_ozone_per_ch4_flux,ozone_rf_per_ch4_flux,h2o_rf_per_ch4_flux,aerosol_rf_per_ch4_flux
GFDL-emi,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
GFDL,41.4,4.39,0.01,16.0,18.8,0.81,0.48,-0.01
INCA,47.3,3.83,0.01,11.3,5.29,0.47,0.14,0.06
OsloCTM,50.3,3.61,0.01,16.5,16.4,0.82,0.20,-0.02
OsloCTM-emi,50.3,3.61,0.01,16.5,16.4,0.82,0.20,-0.02
UKCA,51.8,3.49,0.01,13.6,nan,0.47,nan,nan
WACCM,50.3,3.60,0.02,17.4,15.9,0.85,0.07,nan


In [92]:
frac = df_per_flux_h2_combined['ch4_flux_per_h2_flux'] #Tg CH4/Tg H2 /Tg CH4 = 1/Tg H2
frac['GFDL-emi'] = 0.0
frac

GFDL-emi      0.00
GFDL          0.26
INCA          0.29
OsloCTM       0.31
OsloCTM-emi   0.32
UKCA          0.38
WACCM         0.33
Name: ch4_flux_per_h2_flux, dtype: float64

In [93]:
df_per_flux_ch4_add
#Keep the following:
#deltaH2
#surf_h2_per_h2_flux keep as h2_flux_per_ch4_flux small
#surf_ch4_per_h2_flux
#ch4_flux_per_h2_flux
#ch4_rf_per_h2_flux

#add:
#trop_du_ozone_per_h2_flux
#strat_du_ozone_per_h2_flux
#ozone_rf_per_h2_flux
#h2o_rf_per_h2_flux
#aerosol_rf_per_h2_flux

df_per_flux_h2_combined['trop_du_ozone_per_h2_flux'] = df_per_flux_h2_combined['trop_du_ozone_per_h2_flux'] + df_per_flux_ch4_add['trop_du_ozone_per_ch4_flux']*frac
df_per_flux_h2_combined['strat_du_ozone_per_h2_flux'] = df_per_flux_h2_combined['strat_du_ozone_per_h2_flux'] + df_per_flux_ch4_add['strat_du_ozone_per_ch4_flux']*frac
df_per_flux_h2_combined['ozone_rf_per_h2_flux'] = df_per_flux_h2_combined['ozone_rf_per_h2_flux'] + df_per_flux_ch4_add['ozone_rf_per_ch4_flux']*frac
df_per_flux_h2_combined['h2o_rf_per_h2_flux'] = df_per_flux_h2_combined['h2o_rf_per_h2_flux'] + df_per_flux_ch4_add['h2o_rf_per_ch4_flux']*frac
df_per_flux_h2_combined['aerosol_rf_per_h2_flux'] = df_per_flux_h2_combined['aerosol_rf_per_h2_flux'] + df_per_flux_ch4_add['aerosol_rf_per_ch4_flux']*frac


#df_per_flux_h2_combined

#Save to file:
df_per_flux_h2_combined = df_per_flux_h2_combined.sort_index()
df_per_flux_h2_combined.to_csv(outputpath + 'table_per_flux_h2_combined.csv')


In [94]:
#Rename the columns:
columns_names={'deltaH2':'Flux H2 [Tg/yr]',
               'surf_h2_per_h2_flux': 'Surf. conc. H2 per flux [ppb yr/Tg]',
               'surf_ch4_per_h2_flux':'Surf. conc. CH4 per flux [ppb yr/Tg]',
               'ch4_flux_per_h2_flux':'Flux CH4/Flux H2 [Tg CH4/Tg H2]',
               'ch4_rf_per_h2_flux':'CH4 RF per flux [mW m-2 yr/ Tg]',
               'trop_du_ozone_per_h2_flux':'Trop. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'strat_du_ozone_per_h2_flux':'Strat. ozone per flux [10$^{-3}$ DU yr/Tg]',
               'ozone_rf_per_h2_flux':'ozone RF per flux [mW m-2 yr/ Tg]',
               'h2o_rf_per_h2_flux':'Strat. H2O RF per flux [mW m-2 yr/ Tg]',
               'aerosol_rf_per_h2_flux':'Aerosol RF per flux [mW m-2 yr/ Tg]'}
#Rename column names:
df_per_flux_h2_combined.rename(columns=dict(columns_names),inplace=True) #[df_per_flux_h2.columns])
df_per_flux_h2_combined.rename(model_dict,axis=0,inplace=True)
df_per_flux_h2_combined.loc['Model mean'] = df_per_flux_h2_combined.drop(['GFDL-emi','OsloCTM-emi']).mean()
df_per_flux_h2_combined['Flux H2 [Tg/yr]'].loc['Model mean']=np.nan
df_per_flux_h2_combined=df_per_flux_h2_combined.reindex(sorted_array_2)
df_per_flux_h2_combined

,Flux H2 [Tg/yr],Surf. conc. H2 per flux [ppb yr/Tg],Surf. conc. CH4 per flux [ppb yr/Tg],Flux CH4/Flux H2 [Tg CH4/Tg H2],CH4 RF per flux [mW m-2 yr/ Tg],Trop. ozone per flux [10$^{-3}$ DU yr/Tg],Strat. ozone per flux [10$^{-3}$ DU yr/Tg],ozone RF per flux [mW m-2 yr/ Tg],Strat. H2O RF per flux [mW m-2 yr/ Tg],Aerosol RF per flux [mW m-2 yr/ Tg]
GFDL,30.5,6.98,1.14,0.26,0.44,8.27,2.47,0.42,0.29,-0.01
OsloCTM,8.54,6.23,1.13,0.31,0.43,9.64,5.91,0.47,0.24,-0.04
INCA,7.23,7.36,1.13,0.29,0.43,7.24,-4.01,0.28,0.16,0.05
UKCA,6.87,7.80,1.33,0.38,0.51,9.81,nan,0.34,nan,nan
WACCM,9.93,5.36,1.19,0.33,0.46,9.06,4.14,0.46,0.08,nan
GFDL-emi,197,6.96,1.00,0.00,0.39,7.97,5.23,0.39,0.25,-0.07
OsloCTM-emi,13.8,6.48,1.15,0.32,0.44,9.89,6.01,0.48,0.24,-0.04
Model mean,nan,6.74,1.18,0.32,0.46,8.80,2.13,0.40,0.19,0.00


## H2 budget table

In [95]:
df_budget_h2 = pd.concat([df_h2_burden.loc['CTRL'],
                          df_h2_atmloss.loc['CTRL'],
                          df_h2_atmprod.loc['CTRL'],
                          df_h2_drydep.loc['CTRL'],
                          df_h2_estemis.loc['CTRL'], 
                          df_h2_atm_lifetime.loc['CTRL'],
                          df_h2_soil_sink_lifetime.loc['CTRL'],
                          df_h2_lifetime.loc['CTRL']
                         ],axis=1)

df_budget_h2.columns = ['H2 burden [Tg]',
                        'H2 atm loss [Tg/yr]',
                        'H2 atm prod [Tg/yr]',
                        'H2 soil sink [Tg/yr]',
                        'H2 estimated emissions [Tg/yr]',
                        'H2 atm lifetime [yrs]',
                        'H2 soil sink lifetime [yrs]',
                        'H2 total lifetime [yrs]',]
df_budget_h2 = df_budget_h2.sort_index()
df_budget_h2.to_csv(outputpath + 'table_budget_h2.csv')
df_budget_h2



,H2 burden [Tg],H2 atm loss [Tg/yr],H2 atm prod [Tg/yr],H2 soil sink [Tg/yr],H2 estimated emissions [Tg/yr],H2 atm lifetime [yrs],H2 soil sink lifetime [yrs],H2 total lifetime [yrs]
GFDL-emi,192,22.3,45.1,54.6,31.9,8.62,3.52,2.50
GFDL_nudge,188,21.7,45.0,56.6,33.3,8.66,3.32,2.40
INCA,195,22.5,47.1,52.0,27.3,8.66,3.74,2.61
OSLOCTM3,196,28.4,56.3,59.5,31.6,6.91,3.30,2.23
OSLOCTM3-emi,209,30.1,56.2,58.4,32.2,6.95,3.58,2.36
UKCA,191,26.8,48.4,44.3,22.7,7.15,4.32,2.69
WACCM6-2deg,195,29.0,33.9,73.0,68.1,6.70,2.67,1.91


## CH4 budget table

In [96]:
df_budget_ch4 = pd.concat([df_ch4_burden.loc['CTRL'],
                           df_ch4_surfconc.loc['CTRL'],
                           df_ch4_loss.loc['CTRL'],
                           df_ch4_loss_other_strat.loc['CTRL'],
                           df_ch4_loss_soil.loc['CTRL'],
                           df_ch4_loss.loc['CTRL']+df_ch4_loss_other_strat.loc['CTRL']+df_ch4_loss_soil.loc['CTRL'],
                           df_ch4_lifetime.loc['CTRL'],
                           df_ch4_tot_lifetime.loc['CTRL']
                           ],
                          axis=1)
df_budget_ch4 = df_budget_ch4.sort_index()


df_budget_ch4.columns = ['CH4 burden [Tg]','CH4 surface conc. [ppbv]',
                         'CH4 chem loss OH [Tg/yr]', 'CH4 chem loss other strat [Tg/yr]','CH4 loss soil [Tg/yr]',
                         'CH4 total loss [Tg/yr]',
                         'CH4 lifetime due to OH (whole atmosphere) [yrs]','Total CH4 lifetime [yrs]'] 

df_budget_ch4.to_csv(outputpath + 'table_budget_ch4.csv')

df_budget_ch4

,CH4 burden [Tg],CH4 surface conc. [ppbv],CH4 chem loss OH [Tg/yr],CH4 chem loss other strat [Tg/yr],CH4 loss soil [Tg/yr],CH4 total loss [Tg/yr],CH4 lifetime due to OH (whole atmosphere) [yrs],Total CH4 lifetime [yrs]
GFDL-emi,"4,955","1,832",540,20.6,31.0,592,9.17,8.37
GFDL_nudge,"4,876","1,816",528,20.3,30.5,578,9.24,8.43
INCA,"5,003","1,813",593,20.8,31.3,645,8.44,7.76
OSLOCTM3,"4,995","1,813",683,20.8,31.2,735,7.31,6.80
OSLOCTM3-emi,"4,995","1,813",681,20.8,31.2,733,7.33,6.81
UKCA,"4,939","1,806",624,20.6,30.9,675,7.92,6.61
WACCM6-2deg,"5,003","1,808",727,20.8,31.3,779,6.88,6.42


In [97]:
#Write AGWP values to file
df_h2_agwp  = pd.concat([df_h2_agwp_ch4,
                         df_h2_agwp_o3,
                         df_h2_agwp_h2o,
                         df_h2_agwp_ch4ind_o3,
                         df_h2_agwp_ch4ind_h2o],axis=1,sort=False)

df_h2_agwp.to_csv(outputpath + 'table_h2_agwp.csv') 
df_h2_agwp

,h2_agwp_ch4,h2_agwp_o3,h2_agwp_h2o,h2_agwp_ch4ind_o3,h2_agwp_ch4ind_h2o
OSLOCTM3,0.43,0.22,0.17,0.25,0.06
WACCM6-2deg,0.46,0.18,0.05,0.28,0.02
INCA,0.43,0.15,0.11,0.14,0.04
GFDL-emi,0.39,0.39,0.25,nan,nan
OSLOCTM3-emi,0.44,0.22,0.18,0.26,0.06
UKCA,0.51,0.17,nan,0.18,nan
GFDL_nudge,0.44,0.21,0.16,0.21,0.13


# Appendix:

## Methane feedback factor:

### Atmospheric mass conversion CH4  [Tg/ppb] (from perturbations)

In [98]:
df_ch4_burden_per_conc  = df_ch4_burden.loc['deltaCH4']/df_ch4_surfconc.loc['deltaCH4']
df_ch4_burden_per_conc.name = 'ch4_burden_per_conc'
df_ch4_burden_per_conc

OSLOCTM3       2.76
WACCM6-2deg    2.78
INCA           2.76
GFDL-emi       2.71
OSLOCTM3-emi    nan
UKCA           2.74
GFDL_nudge     2.69
Name: ch4_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [99]:
df_w_o_feedback =df_ch4_tot_lifetime.loc['CTRL']/df_ch4_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback['GFDL-emi']=np.nan
df_w_o_feedback = df_w_o_feedback.sort_index()
df_w_o_feedback.name = 'increase_w_o_feedback'
df_w_o_feedback.to_csv(outputpath + 'increase_w_o_feedback.csv')
df_w_o_feedback

GFDL-emi        nan
GFDL_nudge     3.14
INCA           2.81
OSLOCTM3       2.46
OSLOCTM3-emi    nan
UKCA           2.41
WACCM6-2deg    2.31
Name: increase_w_o_feedback, dtype: float64

### Feedback factor: increase CH4 with feedback/ increase CH4 without feedback

In [100]:
df_feedback_factor_ch4 = df_surf_ch4_per_ch4_flux/df_w_o_feedback
df_feedback_factor_ch4.name = 'feedback_factor_ch4'
df_feedback_factor_ch4 = df_feedback_factor_ch4.sort_index()
df_feedback_factor_ch4['GFDL-emi']=np.nan
df_feedback_factor_ch4.to_csv(outputpath + 'feedback_factor_ch4.csv')
df_feedback_factor_ch4

GFDL-emi        nan
GFDL_nudge     1.40
INCA           1.36
OSLOCTM3       1.47
OSLOCTM3-emi    nan
UKCA           1.45
WACCM6-2deg    1.55
Name: feedback_factor_ch4, dtype: float64

Split the CH4 GWP into direct and indirect based on the feedback factor.

In [101]:
feedback_factor = df_feedback_factor_ch4
feedback_frac = 1.0 - (1.0/feedback_factor)
feedback_frac.name = 'feedback_frac'
feedback_frac

GFDL-emi        nan
GFDL_nudge     0.29
INCA           0.27
OSLOCTM3       0.32
OSLOCTM3-emi    nan
UKCA           0.31
WACCM6-2deg    0.36
Name: feedback_frac, dtype: float64

In [102]:
#Save to file:

df_h2_gwp.loc['CH4dir'] = df_h2_gwp.loc['CH4']*(1.0-feedback_frac)
df_h2_gwp.loc['CH4indir'] = df_h2_gwp.loc['CH4']*feedback_frac

df_h2_gwp['GFDL-emi'].loc['CH4dir'] = df_h2_gwp['GFDL-emi'].loc['CH4']
df_h2_gwp['OSLOCTM3-emi'].loc['CH4dir'] = df_h2_gwp['OSLOCTM3-emi'].loc['CH4']
df_h2_gwp = df_h2_gwp.drop(['total','CH4'])
df_h2_gwp.to_csv(outputpath + 'table_h2_gwp.csv')

df_h2_gwp

/home/ragnhibs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ragnhibs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/ragnhibs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  

,GFDL-emi,GFDL_nudge,INCA,OSLOCTM3,OSLOCTM3-emi,UKCA,WACCM6-2deg
O3,4.34,2.34,1.64,2.44,2.49,1.85,1.97
strat H2O,2.80,1.80,1.27,1.94,1.97,nan,0.58
O3 CH4ind,nan,2.35,1.53,2.85,2.91,2.00,3.13
strat H2O CH4ind,nan,1.41,0.46,0.70,0.72,nan,0.28
CH4dir,4.30,3.52,3.56,3.31,4.96,3.97,3.29
CH4indir,nan,1.40,1.29,1.54,nan,1.77,1.82


In [103]:
df_feedback_factor_ch4

GFDL-emi        nan
GFDL_nudge     1.40
INCA           1.36
OSLOCTM3       1.47
OSLOCTM3-emi    nan
UKCA           1.45
WACCM6-2deg    1.55
Name: feedback_factor_ch4, dtype: float64

In [104]:
#Alternative feedback calc:
#ss the sensitivity of the lifetime to the burden.

#ss = 1 – dln[<k All><H2>]/dln[<H2>] 
#= 1 – dln[<k All>]/dln[<H2>] – dln[<H2>]/dln[<H2>] 
#= – dln[<k All>]/dln[<H2>] 
#=  +dln[1/<k All>] / dln[<H2>] 

#<k All> = <k OH> + <kstrat OH> + <k soil> Are the inverse liftime-
k_all_inv = df_ch4_tot_lifetime

a = np.log(k_all_inv.loc['10CH4'])-np.log(k_all_inv.loc['CTRL'])

b = np.log(df_ch4_burden.loc['10CH4'])-np.log(df_ch4_burden.loc['CTRL'])
#print(b)
ss = a/b
ff = 1/(1-ss)
print(ff)

OSLOCTM3       1.44
WACCM6-2deg    1.53
INCA           1.35
GFDL-emi       19.8
OSLOCTM3-emi    nan
UKCA           1.43
GFDL_nudge     1.38
dtype: float64


In [105]:
ss = (np.log(df_ch4_tot_lifetime.loc['10CH4']/df_ch4_tot_lifetime.loc['CTRL']))/np.log(df_ch4_burden.loc['10CH4']/df_ch4_burden.loc['CTRL'])
ff=1/(1-ss)
print(ff.drop('GFDL-emi'))

OSLOCTM3       1.44
WACCM6-2deg    1.53
INCA           1.35
OSLOCTM3-emi    nan
UKCA           1.43
GFDL_nudge     1.38
dtype: float64


## Hydrogen feedback factor:

### Atmospheric mass conversion H2  [Tg/ppb] (from perturbations)

In [106]:
df_h2_burden_per_conc  = df_h2_burden.loc['deltaH2']/df_h2_surfconc.loc['deltaH2']
df_h2_burden_per_conc.name = 'h2_burden_per_conc'
df_h2_burden_per_conc

OSLOCTM3       0.35
WACCM6-2deg    0.35
INCA           0.35
GFDL-emi       0.34
OSLOCTM3-emi   0.35
UKCA           0.34
GFDL_nudge     0.34
Name: h2_burden_per_conc, dtype: float64

### Increase per unit flux w/o feedback = integrated decay [ppb yr/Tg]

In [107]:
df_w_o_feedback_h2 =df_h2_lifetime.loc['CTRL']/df_h2_burden_per_conc #Lifetime [yr] / [Tg/ppb] 
df_w_o_feedback_h2

OSLOCTM3       6.36
WACCM6-2deg    5.44
INCA           7.44
GFDL-emi       7.34
OSLOCTM3-emi   6.78
UKCA           7.82
GFDL_nudge     7.01
dtype: float64

### Feedback factor: increase H2 with feedback/ increase H2 without feedback

If you take the control lifetime from the budget terms and divide it into the lifetime of the perturbation (the added burden from the 10% increase divided by the change in chemical flux from that perturbation.) You should get the feed back factor.

In [108]:
df_feedback_factor_h2 = df_surf_h2_per_h2_flux/df_w_o_feedback_h2
df_feedback_factor_h2.name = 'feedback_factor_h2'
df_feedback_factor_h2
#Fabien wrote in the paper about feedback factor less than 1.

OSLOCTM3       0.98
WACCM6-2deg    0.99
INCA           0.99
GFDL-emi       0.95
OSLOCTM3-emi   0.96
UKCA           1.00
GFDL_nudge     1.00
Name: feedback_factor_h2, dtype: float64

### Change in lifetime per flux

In [109]:
df_ch4_lifetime.loc['deltaH2'] = df_ch4_lifetime.loc['10H2']-df_ch4_lifetime.loc['CTRL']
df_ch4_lifetime.loc['deltaCH4'] = df_ch4_lifetime.loc['10CH4']-df_ch4_lifetime.loc['CTRL']
df_ch4_lifetime

,OSLOCTM3,WACCM6-2deg,INCA,GFDL-emi,OSLOCTM3-emi,UKCA,GFDL_nudge
Scenario,,,,,,,
CTRL,7.31,6.88,8.44,9.17,7.33,7.92,9.24
10H2,7.34,6.91,8.47,9.93,7.38,7.95,9.38
10CH4,7.55,7.13,8.67,10.2,nan,8.19,9.51
deltaH2,0.03,0.03,0.03,0.77,0.05,0.03,0.14
deltaCH4,0.23,0.25,0.23,1.05,nan,0.27,0.27


In [110]:
#Direct (changes in methane lifetime per h2 flux [days per Tg H2])
df_ch4_lifetime_per_h2_flux =  df_ch4_lifetime.loc['deltaH2']/df_h2_flux.loc['deltaH2']
df_ch4_lifetime_per_h2_flux*365.0 #Days

OSLOCTM3       1.23
WACCM6-2deg    1.15
INCA           1.54
GFDL-emi       1.42
OSLOCTM3-emi   1.27
UKCA           1.86
GFDL_nudge     1.69
Name: deltaH2, dtype: float64

In [111]:
#Indirect (changes in methane lifetime per h2 flux [days per Tg H2] due to changes in methane):
df_ch4_lifetime_per_ch4_flux =  df_ch4_lifetime.loc['deltaCH4']/df_ch4_flux.loc['deltaCH4']
df_ch4_lifetime_per_ch4_flux*365.0*df_ch4_flux_per_h2_flux


OSLOCTM3       0.53
WACCM6-2deg    0.60
INCA           0.52
GFDL-emi       26.2
OSLOCTM3-emi    nan
UKCA           0.72
GFDL_nudge     0.62
dtype: float64

NB be aware that the 10CH4 in GFDL-emi is not the methane experiment as done in the other experiments!

### Check that delta flux equal delta burden divided by lifetime including the feedback effect

In [112]:
df_feedback_factor_h2

OSLOCTM3       0.98
WACCM6-2deg    0.99
INCA           0.99
GFDL-emi       0.95
OSLOCTM3-emi   0.96
UKCA           1.00
GFDL_nudge     1.00
Name: feedback_factor_h2, dtype: float64

In [113]:
ss = (np.log(df_h2_lifetime.loc['10H2']/df_h2_lifetime.loc['CTRL']))/np.log(df_h2_burden.loc['10H2']/df_h2_burden.loc['CTRL'])
ff=1/(1-ss)
print(ff.drop('GFDL-emi'))

OSLOCTM3       0.98
WACCM6-2deg    0.99
INCA           0.99
OSLOCTM3-emi   0.96
UKCA           1.00
GFDL_nudge     1.00
dtype: float64


In [114]:
#Delta burden divided by tau*ff
test = df_h2_burden.loc['deltaH2']/(df_h2_lifetime.loc['CTRL']*df_feedback_factor_h2)
print(test)
print(df_h2_flux.loc['deltaH2'])


OSLOCTM3       8.54
WACCM6-2deg    9.93
INCA           7.23
GFDL-emi        197
OSLOCTM3-emi   13.8
UKCA           6.87
GFDL_nudge     30.5
dtype: float64
OSLOCTM3       8.54
WACCM6-2deg    9.93
INCA           7.23
GFDL-emi        197
OSLOCTM3-emi   13.8
UKCA           6.87
GFDL_nudge     30.5
Name: deltaH2, dtype: float64


In [115]:
test = df_ch4_burden.loc['deltaCH4']/(df_ch4_tot_lifetime.loc['CTRL']*df_feedback_factor_ch4)
print(test.sort_index())
print(df_ch4_flux.loc['deltaCH4'].sort_index())

GFDL-emi        nan
GFDL_nudge     41.4
INCA           47.3
OSLOCTM3       50.3
OSLOCTM3-emi    nan
UKCA           51.8
WACCM6-2deg    50.3
dtype: float64
GFDL-emi       3.10
GFDL_nudge     41.4
INCA           47.3
OSLOCTM3       50.3
OSLOCTM3-emi   50.3
UKCA           51.8
WACCM6-2deg    50.3
Name: deltaCH4, dtype: float64


# Model mean uncertainty

In [116]:
tau = np.mean(df_h2_lifetime.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'], axis=1).loc[['CTRL']].values*df_feedback_factor_h2.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).values)
sigma_agwp100CO2 = 0.26/2*agwp100_CO2
#tau = np.mean(df_h2_lifetime.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'], axis=1).loc[['CTRL']].values)
#print(df_h2_atmloss)
#print(df_ozone_rf)
#print(df_h2o_rf)
#print(df_h2_agwp_o3)
gfdl_scaled_burden = df_h2_burden.loc['deltaH2']['GFDL_nudge']/4
burden_for_uncertainties = np.append(df_h2_burden.drop(labels=['GFDL-emi', 'OSLOCTM3-emi', 'GFDL_nudge'], axis=1).loc[['deltaH2']].values,gfdl_scaled_burden)
sigma_burden = np.std(burden_for_uncertainties)
#sigma_burden = np.std(df_h2_burden.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'], axis=1).loc[['deltaH2']].values/df_h2_burden.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'], axis=1).mean(axis = 1).loc[['CTRL']].values)
sigma_drydep = 15#np.std(df_h2_drydep.loc[['CTRL']].values)
sigma_atmloss = np.std(df_h2_atmloss.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'], axis=1).loc[['CTRL']].values)
sigma_rf_methane = 0.20
sigma_ff_ch4 = 0.07

print(df_per_flux_ch4.drop(labels=['GFDL-emi', 'OsloCTM-emi', 'Model mean'], axis=0).mean(axis = 1).values/df_feedback_factor_ch4.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).values)
sigma_delta_flux_CH4_dir = np.std(df_per_flux_ch4.drop(labels=['GFDL-emi', 'OsloCTM-emi', 'Model mean'], axis=0).mean(axis = 1).values/df_feedback_factor_ch4.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).values)
#sigma_delta_flux_CH4_dir = np.std(df_per_flux_ch4.drop(labels=['GFDL-emi', 'OsloCTM-emi'], axis=0).mean(axis = 1).values)
sigma_rf_o3 = np.std((df_h2_agwp_o3.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'])+df_h2_agwp_ch4ind_o3.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']))/tau)
sigma_rf_h2o = np.std((df_h2_agwp_h2o.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'])+ df_h2_agwp_ch4ind_o3.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']))/tau)



[7.31679707 8.05182063 5.83351353 9.59182266 8.09565555]


In [117]:
print(sigma_drydep)
print(sigma_burden)
print(sigma_atmloss)
print(sigma_agwp100CO2)
print(burden_for_uncertainties)
sigma_h2_ff = np.std(df_feedback_factor_h2.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']))

15
0.1732134287663628
3.028206428021064
0.011635
[18.67713195 18.6634     18.6821     18.4208     18.254375  ]


In [118]:
gwp = df_h2_gwp.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'], axis=1).mean(axis = 1).sum()
print(gwp)

delta_burden = np.mean(burden_for_uncertainties)
loss = np.mean(df_h2_loss.drop(labels=['GFDL-emi', 'OSLOCTM3-emi'], axis=1).loc[['CTRL']].values)
print(delta_burden)
print(loss)
print(tau)
delta_flux_CH4 = df_per_flux_ch4.drop(labels=['GFDL-emi', 'OsloCTM-emi'], axis=0)['Surf. conc. CH4 per flux [ppb yr/Tg]'].mean()
print(delta_flux_CH4)
rf_o3_m = df_h2_agwp_o3.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).mean()/tau
rf_o3_indir_m = df_h2_agwp_ch4ind_o3.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).mean()/tau
rf_h2o_m = df_h2_agwp_h2o.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).mean()/tau
rf_h2o_indir_m = df_h2_agwp_ch4ind_h2o.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).mean()/tau
ff_ch4 = df_feedback_factor_ch4.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).mean()
ff_h2 = df_feedback_factor_h2.drop(labels=['GFDL-emi', 'OSLOCTM3-emi']).mean()
delta_flux_CH4_dir = delta_flux_CH4*1.0/ff_ch4
print(ff_ch4)
print(df_h2_agwp_o3)

11.621194154700564
18.539561389333333
82.74235077080513
2.3453563763446232
3.782100645641515
1.4458555503863806
OSLOCTM3       0.22
WACCM6-2deg    0.18
INCA           0.15
GFDL-emi       0.39
OSLOCTM3-emi   0.22
UKCA           0.17
GFDL_nudge     0.21
Name: h2_agwp_o3, dtype: float64


In [119]:
scaled_sigma_methane = spec_rf_ch4**2*delta_flux_CH4**2*((sigma_rf_methane)**2 + (sigma_ff_ch4/ff_ch4)**2 + (sigma_delta_flux_CH4_dir/delta_flux_CH4_dir)**2)
sigma = gwp*np.sqrt((sigma_agwp100CO2/agwp100_CO2)**2 + + (sigma_h2_ff/ff_h2)**2+ (sigma_burden/delta_burden)**2 +1/(loss)**2*(sigma_drydep**2 + sigma_atmloss**2) + (tau*agwp100_CO2/gwp)**2*( scaled_sigma_methane + sigma_rf_h2o**2 + sigma_rf_o3**2))
print(sigma)
print(sigma/gwp)
print(sigma_rf_methane*spec_rf_ch4)
print(sigma_h2_ff)

2.6350359108968786
0.22674398825279535
0.07705600000000001
0.006313861044479247


In [120]:
print(f"Uncertainty AGWP_CO2: {sigma_agwp100CO2/agwp100_CO2}")
print(f"Uncertainty burden: {sigma_burden/delta_burden}")
print(f"Uncertainty drydep: {sigma_drydep/loss}")
print(f"Uncertainty atmospheric loss: {sigma_atmloss/loss}")
print(f"Uncertainty RE methane: {sigma_rf_methane*delta_flux_CH4*tau*agwp100_CO2/gwp}")
print(f"Uncertainty delta feedback factor methane: {sigma_ff_ch4*delta_flux_CH4_dir*spec_rf_ch4*tau*agwp100_CO2/gwp}")
print(f"Uncertainty delta flux methane: {sigma_delta_flux_CH4_dir*ff_ch4*spec_rf_ch4*tau*agwp100_CO2/gwp}")
print(f"Uncertainty stratospheric water vapour: {sigma_rf_h2o*tau*agwp100_CO2/gwp}")
print(f"Uncertainty stratospheric ozone: {sigma_rf_o3*tau*agwp100_CO2/gwp}")
print(f"Uncertainty feedback factor of hydrogen: {sigma_h2_ff/ff_h2}")

Uncertainty AGWP_CO2: 0.13
Uncertainty burden: 0.009342908665898672
Uncertainty drydep: 0.18128563982367069
Uncertainty atmospheric loss: 0.03659802265479673
Uncertainty RE methane: 0.01366294118091446
Uncertainty delta feedback factor methane: 0.0012742768749420352
Uncertainty delta flux methane: 0.01229007186909054
Uncertainty stratospheric water vapour: 0.0004960323853942994
Uncertainty stratospheric ozone: 0.0005488121539335447
Uncertainty feedback factor of hydrogen: 0.006381269506848704


In [121]:
df_needed_for_uncertainties = pd.DataFrame(data={"delta_burden":delta_burden, "sigma_burden":sigma_burden, "loss":loss, "sigma_atmloss":sigma_atmloss, "sigma_h2_ff":sigma_h2_ff, "ff_h2":ff_h2, "tau":tau, "ff_ch4":ff_ch4, "delta_flux_CH4":delta_flux_CH4, "sigma_delta_flux_CH4_dir":sigma_delta_flux_CH4_dir}, index = [0])
print(df_needed_for_uncertainties)
df_needed_for_uncertainties.to_csv(outputpath + 'uncertainty_input.txt')

   delta_burden  sigma_burden  loss  sigma_atmloss  sigma_h2_ff  ff_h2  tau  \
0          18.5          0.17  82.7           3.03         0.01   0.99 2.35   

   ff_ch4  delta_flux_CH4  sigma_delta_flux_CH4_dir  
0    1.45            3.78                      1.22  
